In [ ]:
!pip install -r requirements.txt
# This script installs the required packages for the Aero-delay prediction project.
# Make sure to run this in an environment where you have the necessary permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd

/Users/harish/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("patrickzel/flight-delay-and-cancellation-dataset-2019-2023")

print("Path to dataset files:", path)

/Users/harish/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 140M/140M [00:27<00:00, 5.43MB/s] 

Extracting files...


Path to dataset files: /Users/harish/.cache/kagglehub/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023/versions/7


## Load Dataset and filter to dates after `2022-07-31` to speed up compute

In [18]:
df = pd.read_csv("../data/raw/flights_sample_3m.csv")

In [15]:
df.head()

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


In [19]:
df.shape

(3000000, 32)

In [20]:
df_reduced = df[df['FL_DATE'] >= '2022-07-31']

In [31]:
df_reduced.shape

(754303, 33)

In [32]:
df_reduced.drop(columns=['Unnamed: 0'], inplace=True)

In [33]:
df_reduced.head()

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
1,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
2,2023-06-11,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,2134,DCA,"Washington, DC",BOS,"Boston, MA",...,0.0,109.0,89.0,58.0,399.0,NaN,NaN,NaN,NaN,NaN
3,2023-02-12,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,590,IAH,"Houston, TX",LAX,"Los Angeles, CA",...,0.0,227.0,219.0,200.0,1379.0,NaN,NaN,NaN,NaN,NaN
4,2022-11-12,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2706,GRR,"Grand Rapids, MI",MSP,"Minneapolis, MN",...,0.0,96.0,76.0,61.0,408.0,NaN,NaN,NaN,NaN,NaN


In [34]:
## Now save this back
df_reduced.to_csv("../data/raw/flights_sample_reduced.csv", index=True)

## Begin exploring data

In [33]:
df_reduced = pd.read_csv("../data/raw/flights_sample_reduced.csv")

In [34]:
# examine the data
df_reduced.head()

,Unnamed: 0,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,0,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
1,1,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
2,2,2023-06-11,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,2134,DCA,"Washington, DC",BOS,...,0.0,109.0,89.0,58.0,399.0,NaN,NaN,NaN,NaN,NaN
3,3,2023-02-12,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,590,IAH,"Houston, TX",LAX,...,0.0,227.0,219.0,200.0,1379.0,NaN,NaN,NaN,NaN,NaN
4,4,2022-11-12,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2706,GRR,"Grand Rapids, MI",MSP,...,0.0,96.0,76.0,61.0,408.0,NaN,NaN,NaN,NaN,NaN


In [35]:
# examine the columns
df_reduced.columns

Index(['Unnamed: 0', 'FL_DATE', 'AIRLINE', 'AIRLINE_DOT', 'AIRLINE_CODE',
       'DOT_CODE', 'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'DELAY_DUE_CARRIER',
       'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY',
       'DELAY_DUE_LATE_AIRCRAFT'],
      dtype='object')

In [41]:
## try to add weather dataso
df_reduced.shape

(754303, 37)

In [37]:
# clean weather from .dat to .csv
import pandas as pd

# OpenFlights columns
cols = [
    "airport_id","name","city","country","IATA","ICAO",
    "lat","lon","alt","timezone","dst","tz_db","type","source"
]
air = pd.read_csv("../data/raw/airports.dat", names=cols)

# Keep only needed columns & drop bad rows
air = air[["IATA","ICAO","name","lat","lon","tz_db"]]
air = air.dropna(subset=["IATA","lat","lon","tz_db"])
air = air[air["IATA"] != "\\N"].drop_duplicates("IATA")

# Save cleaned CSV
air.to_csv("../data/raw/airports.csv", index=False)

print(f"Saved {len(air)} airports to data/raw/airports.csv")

Saved 6072 airports to data/raw/airports.csv


In [43]:
def add_delay_flags_simple(df):
    """
    Remove canceled/diverted flights and add binary delay flags.
    delay_15 = 1 if ARR_DELAY >= 15 minutes
    delay_30 = 1 if ARR_DELAY >= 30 minutes
    """
    df = df.copy()

    # Filter out canceled or diverted flights
    df = df[(df["CANCELLED"] == 0) & (df["DIVERTED"] == 0)]

    # Create delay flags
    df["delay_15"] = (df["ARR_DELAY"] >= 15).astype(int)
    df["delay_30"] = (df["ARR_DELAY"] >= 30).astype(int)

    return df

# --- usage ---
# df_clean = add_delay_flags_simple(df_reduced)

In [44]:
df_clean = add_delay_flags_simple(df_reduced)

In [45]:
df_clean.shape

(738053, 37)

✈️ Flight Weather Enrichment Pipeline — High-Level Steps

1. Input & Requirements
	•	Takes a flights dataset (BTS-style) with:
	•	FL_DATE, ORIGIN, DEST, CRS_DEP_TIME, CRS_ARR_TIME
	•	Needs an airports CSV mapping IATA codes → latitude & longitude.

⸻

2. Create Local Time Keys
	•	Convert CRS_DEP_TIME & CRS_ARR_TIME (HHMM format) into hourly timestamps in local flight date.
	•	Two keys created:
	•	dep_hour_dt → planned departure hour
	•	arr_hour_dt → planned arrival hour

⸻

3. Build Unique Weather Requests
	•	Combine all (airport, hour) pairs from dep & arr flights.
	•	Extract year from each timestamp.
	•	Build a list of (IATA, year) pairs → minimum needed weather calls.

⸻

4. Fetch & Cache Raw Weather Data
	•	For each (IATA, year):
	1.	Check local cache (weather_cache/IATA_YEAR_RAW.parquet).
	2.	If cached → load it.
	3.	If not:
	•	Use Meteostat Hourly API to fetch:
	•	temp (°C), prcp (mm), wspd (km/h)
	•	Save raw data to cache (keeps API usage low).
	•	Uses multi-threading to fetch multiple airports in parallel.
	•	Shows a progress bar while fetching.

⸻

5. Compute Weather Flags

From the raw weather data, create binary flags:
	•	rain → precipitation > 0
	•	ice  → temperature ≤ 0°C and precipitation > 0
	•	wind → wind speed ≥ wind_kmh_threshold (user-configurable)

Because the cache stores raw data, you can change the threshold and recompute without re-fetching.

⸻

6. Merge Weather into Flights
	•	For departures: join on (ORIGIN, dep_hour_dt)
	•	For arrivals: join on (DEST, arr_hour_dt)
	•	Add:
	•	dep_rain, dep_ice, dep_wind
	•	arr_rain, arr_ice, arr_wind
	•	Also add *_missing flags to mark where weather data wasn’t available.

⸻

7. Output
	•	Returns the original flights DataFrame with 6 weather flags + 6 missing flags added.
	•	Efficient (fetches once per airport/year), reusable (cache), and configurable.

⸻


In [46]:
def enrich_with_weather_flags(
    df_flights,
    airports_csv="../data/raw/airports.csv",  # Must contain IATA, lat, lon
    cache_dir="weather_cache",               # Where to save raw weather data
    wind_kmh_threshold=30,                   # Wind speed threshold for "wind" flag
    max_workers=12,                          # How many threads to fetch weather in parallel
    force_refresh=False                      # If True, ignore cache and re-download
):
    """
    Adds departure & arrival weather flags (rain, ice, wind) to a flight dataset.

    Flags are computed based on the **scheduled local hour** for both departure and arrival:
      - dep key: (ORIGIN airport code, FL_DATE + CRS_DEP_TIME hour)
      - arr key: (DEST   airport code, FL_DATE + CRS_ARR_TIME hour)

    Steps:
      1. Read airport coordinates from CSV.
      2. Convert flight times to datetime hour buckets.
      3. For each (airport, year), fetch hourly weather data from Meteostat.
         - Caches the **raw weather data** to disk for reuse.
      4. Compute weather flags (rain, ice, wind) from cached data.
      5. Merge weather flags back to the flights DataFrame.

    Returns:
      Copy of the flights DataFrame with:
        dep_rain, dep_ice, dep_wind, arr_rain, arr_ice, arr_wind
        and *_missing flags (1 if data missing, 0 otherwise)
    """

    import warnings; warnings.simplefilter("ignore")  # Hide pandas warnings
    import pandas as pd
    from pathlib import Path
    from datetime import datetime
    from meteostat import Hourly, Point
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from tqdm.auto import tqdm

    # ------------------------------------------------------------
    # Helper: Convert HHMM (e.g. 930, 2130) to just the hour (0-23)
    # ------------------------------------------------------------
    def hhmm_to_hour(x):
        if pd.isna(x): 
            return 0
        s = str(int(float(x))).zfill(4)  # 930 -> "0930"
        return int(s[:2])                # "0930" -> 9

    # ------------------------------------------------------------
    # Helper: Add departure & arrival datetime-hour columns
    # ------------------------------------------------------------
    def add_hour_keys(df):
        df = df.copy()
        df["FL_DATE"] = pd.to_datetime(df["FL_DATE"])
        dep_h = df["CRS_DEP_TIME"].apply(hhmm_to_hour)
        arr_h = df["CRS_ARR_TIME"].apply(hhmm_to_hour)
        df["dep_hour_dt"] = pd.to_datetime(df["FL_DATE"].dt.date) + pd.to_timedelta(dep_h, unit="h")
        df["arr_hour_dt"] = pd.to_datetime(df["FL_DATE"].dt.date) + pd.to_timedelta(arr_h, unit="h")
        return df

    # ------------------------------------------------------------
    # Helper: Compute binary weather flags from raw weather data
    # ------------------------------------------------------------
    def compute_flags(raw_df, thr):
        """
        Expects columns: ts (datetime), prcp (mm), temp (°C), wspd (km/h), IATA
        """
        out = raw_df.copy()
        out["rain"] = (out["prcp"].fillna(0) > 0).astype("int8")
        out["ice"]  = ((out["temp"].fillna(99) <= 0) & (out["prcp"].fillna(0) > 0)).astype("int8")
        out["wind"] = (out["wspd"].fillna(0) >= thr).astype("int8")
        return out[["ts", "rain", "ice", "wind", "IATA"]]

    # ------------------------------------------------------------
    # Helper: Fetch hourly weather for a single airport-year
    # ------------------------------------------------------------
    def fetch_airport_year(iata, lat, lon, year):
        """
        Returns hourly weather (ts, prcp, temp, wspd, IATA) for one airport in one year.
        Caches raw data to Parquet in `cache_dir`.
        """
        fpath = CACHE_DIR / f"{iata}_{year}_RAW.parquet"

        # Use cached data if available
        if not force_refresh and fpath.exists():
            return pd.read_parquet(fpath)

        # Download from Meteostat
        p = Point(float(lat), float(lon))
        start = datetime(int(year), 1, 1)
        end   = datetime(int(year), 12, 31, 23, 59, 59)
        wx = Hourly(p, start, end).fetch()  # Meteostat returns UTC timestamps

        if wx.empty:
            raw = pd.DataFrame(columns=["ts", "prcp", "temp", "wspd", "IATA"])
        else:
            raw = wx.reset_index().rename(columns={"time": "ts"})
            raw = raw[["ts", "prcp", "temp", "wspd"]]
            raw["IATA"] = iata

        raw.to_parquet(fpath, index=False)
        return raw

    # ------------------------------------------------------------
    # Step 1: Validate input DataFrame
    # ------------------------------------------------------------
    required_cols = {"FL_DATE", "ORIGIN", "DEST", "CRS_DEP_TIME", "CRS_ARR_TIME"}
    missing_cols = required_cols - set(df_flights.columns)
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # ------------------------------------------------------------
    # Step 2: Prepare flight keys for weather lookup
    # ------------------------------------------------------------
    df = add_hour_keys(df_flights)

    # Load airports CSV with lat/lon
    ap = pd.read_csv(airports_csv)
    if {"IATA", "lat", "lon"} - set(ap.columns):
        raise ValueError("Airports CSV must contain IATA, lat, lon columns")
    ap = ap.dropna(subset=["IATA", "lat", "lon"]).drop_duplicates("IATA").set_index("IATA")[["lat", "lon"]]

    # Keep only flights with known airports
    df = df[df["ORIGIN"].isin(ap.index) & df["DEST"].isin(ap.index)].copy()

    # Create lookup keys for departure and arrival
    dep_keys = df[["ORIGIN", "dep_hour_dt"]].rename(columns={"ORIGIN": "IATA", "dep_hour_dt": "ts"})
    arr_keys = df[["DEST", "arr_hour_dt"]].rename(columns={"DEST": "IATA", "arr_hour_dt": "ts"})
    all_keys = pd.concat([dep_keys, arr_keys], ignore_index=True).dropna().drop_duplicates()
    all_keys["year"] = all_keys["ts"].dt.year

    # Airport-year pairs to fetch
    pairs = all_keys[["IATA", "year"]].drop_duplicates().join(ap, on="IATA", how="inner")

    # ------------------------------------------------------------
    # Step 3: Fetch weather data in parallel
    # ------------------------------------------------------------
    CACHE_DIR = Path(cache_dir)
    CACHE_DIR.mkdir(parents=True, exist_ok=True)

    raw_weather_parts = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {
            ex.submit(fetch_airport_year, row.IATA, row.lat, row.lon, int(row.year)): (row.IATA, int(row.year))
            for row in pairs.itertuples(index=False)
        }
        for fut in tqdm(as_completed(futures), total=len(futures), desc="Fetching Meteostat raw (airport-year)"):
            raw_weather_parts.append(fut.result())

    raw_weather = (
        pd.concat(raw_weather_parts, ignore_index=True)
        if raw_weather_parts else
        pd.DataFrame(columns=["ts", "prcp", "temp", "wspd", "IATA"])
    ).drop_duplicates(subset=["IATA", "ts"])

    # ------------------------------------------------------------
    # Step 4: Compute weather flags & merge to flights
    # ------------------------------------------------------------
    wx_flags = compute_flags(raw_weather, wind_kmh_threshold)

    # Merge departure weather
    dep_weather = (
        df[["ORIGIN", "dep_hour_dt"]]
        .rename(columns={"ORIGIN": "IATA", "dep_hour_dt": "ts"})
        .merge(wx_flags, on=["IATA", "ts"], how="left")
        .rename(columns={"rain": "dep_rain", "ice": "dep_ice", "wind": "dep_wind"})
    )

    # Merge arrival weather
    arr_weather = (
        df[["DEST", "arr_hour_dt"]]
        .rename(columns={"DEST": "IATA", "arr_hour_dt": "ts"})
        .merge(wx_flags, on=["IATA", "ts"], how="left")
        .rename(columns={"rain": "arr_rain", "ice": "arr_ice", "wind": "arr_wind"})
    )

    # Attach columns back
    df["dep_rain"], df["dep_ice"], df["dep_wind"] = dep_weather["dep_rain"].values, dep_weather["dep_ice"].values, dep_weather["dep_wind"].values
    df["arr_rain"], df["arr_ice"], df["arr_wind"] = arr_weather["arr_rain"].values, arr_weather["arr_ice"].values, arr_weather["arr_wind"].values

    # Missing flags
    for col in ["dep_rain", "dep_ice", "dep_wind", "arr_rain", "arr_ice", "arr_wind"]:
        df[f"{col}_missing"] = df[col].isna().astype("int8")
        df[col] = df[col].fillna(0).astype("int8")

    return df

In [51]:
# pip install meteostat tqdm
df_weather = enrich_with_weather_flags(
    df_reduced,
    airports_csv="../data/raw/airports.csv",  # must have IATA,lat,lon
    cache_dir="weather_cache",
    wind_kmh_threshold=35,
    max_workers=12
)
df_weather.head()

Fetching Meteostat raw (airport-year): 100%|██████████| 709/709 [00:00<00:00, 1259.56it/s]


,Unnamed: 0,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,...,dep_wind,arr_rain,arr_ice,arr_wind,dep_rain_missing,dep_ice_missing,dep_wind_missing,arr_rain_missing,arr_ice_missing,arr_wind_missing
0,0,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,...,0,0,0,0,0,0,0,0,0,0
1,1,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,...,0,0,0,0,0,0,0,0,0,0
2,2,2023-06-11,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,2134,DCA,"Washington, DC",BOS,...,0,0,0,0,0,0,0,0,0,0
3,3,2023-02-12,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,590,IAH,"Houston, TX",LAX,...,0,0,0,0,0,0,0,0,0,0
4,4,2022-11-12,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2706,GRR,"Grand Rapids, MI",MSP,...,0,0,0,0,0,0,0,0,0,0


In [52]:
df_weather['dep_wind'].value_counts()

dep_wind
0    743991
1      9917
Name: count, dtype: int64

In [55]:
df_weather_2['arr_rain'].value_counts()

arr_rain
0    696231
1     57677
Name: count, dtype: int64

In [57]:
# List of weather flags
flags = ["dep_rain", "dep_ice", "dep_wind", "arr_rain", "arr_ice", "arr_wind"]

# Calculate counts and percentages
for col in flags:
    counts = df_weather[col].value_counts().sort_index()
    total = counts.sum()
    pct = (counts / total * 100).round(2)
    
    print(f"\n{col}:")
    for val in counts.index:
        print(f"  {val}: {counts[val]} flights ({pct[val]}%)")


dep_rain:
  0: 697463 flights (92.51%)
  1: 56445 flights (7.49%)

dep_ice:
  0: 749722 flights (99.44%)
  1: 4186 flights (0.56%)

dep_wind:
  0: 743991 flights (98.68%)
  1: 9917 flights (1.32%)

arr_rain:
  0: 696231 flights (92.35%)
  1: 57677 flights (7.65%)

arr_ice:
  0: 749972 flights (99.48%)
  1: 3936 flights (0.52%)

arr_wind:
  0: 742837 flights (98.53%)
  1: 11071 flights (1.47%)


In [15]:
from meteostat import Hourly, Point
from datetime import datetime

# Example: Minneapolis–Saint Paul (MSP)
lat, lon = 44.8848, -93.2223
point = Point(lat, lon)
start = datetime(2022, 1, 1)
end   = datetime(2022, 1, 3)  # short range for preview

wx = Hourly(point, start, end).fetch()
print(wx.head())

                     temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  \
time                                                                           
2022-01-01 00:00:00 -12.2 -16.0  73.0   0.3   NaN  350.0  18.4   NaN  1014.8   
2022-01-01 01:00:00 -12.2 -17.3  66.0   0.0   NaN  350.0  16.6   NaN  1016.1   
2022-01-01 02:00:00 -13.3 -19.4  60.0   0.0   NaN  350.0  27.7   NaN  1017.4   
2022-01-01 03:00:00 -14.4 -20.7  59.0   0.0   NaN  360.0  25.9   NaN  1017.6   
2022-01-01 04:00:00 -15.6 -21.0  63.0   0.0   NaN  360.0  20.5   NaN  1018.4   

                     tsun  coco  
time                             
2022-01-01 00:00:00   NaN   4.0  
2022-01-01 01:00:00   NaN   4.0  
2022-01-01 02:00:00   NaN   4.0  
2022-01-01 03:00:00   NaN   3.0  
2022-01-01 04:00:00   NaN   3.0  
